<img src="https://www.rice.edu/_images/rice-logo.jpg" width="200">

# Ensemble Learning - Stacking Linear Regression (SLR)
## The RICE visit - June 17th, 2019

### Software dependencies

In [1]:
#!pip install git+git://github.com/rasbt/mlxtend.git@v0.16.0
import math
import numpy as np
import os
import pandas as pd
import pickle
import random
import seaborn as sns
import time
import warnings
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, Axes3D

# MLXtend (http://rasbt.github.io/mlxtend/)
from mlxtend.regressor import StackingRegressor, StackingCVRegressor

# Scikit-learn (https://scikit-learn.org/stable/)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as Co
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVR

warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_style("whitegrid")


### Illustrative example - Test function and design of experiments

In [2]:
def load_data():
  """
  Data loading and sample splitting
  
  Output: dictionary with training (X, y) and testing (Xtest, ytest) data
  """
  
  # Loading sample dataset - FSAE Romero & Queipo 2017
  print('Loading data...', end=' ')
    
  fsae_df = pd.read_csv('fsae_case_study.csv', delimiter=',', index_col=None, decimal='.')
  (sample_size, dim) = fsae_df.shape
  print(fsae_df.shape)
   
  # Natural Logarithm - Young modulus
  fsae_df.iloc[:,-2] = np.log(fsae_df.iloc[:,-2])
  
  # Normalizing data
  scaler = StandardScaler().fit(fsae_df.iloc[:,0:-1])
  fsae_df.iloc[:,0:-1] = scaler.transform(fsae_df.iloc[:,0:-1])
  
  # Data splitting
  idxs = random.sample(range(len(fsae_df)), k=180)
  X = fsae_df.iloc[idxs,0:-1].values
  y = fsae_df.iloc[idxs,-1].values
  compl_idxs = [i for i in range(len(fsae_df)) if i not in idxs]
  Xtest = fsae_df.iloc[compl_idxs,0:-1].values
  ytest = fsae_df.iloc[compl_idxs,-1].values
    
  return { 'X':X, 'y':y, 'Xtest':Xtest, 'ytest':ytest }

### Grid search definition & model training

In [3]:
def model_train(seed, sample):
  """
  Base learner and SLR definitions, hyper-parameters grid, training
  
  Input: sample - dictionary with training (X, y) data
  Output: dictionary with trained base learners and SLR
  """
  
  # Stacking regressor object
  # SVR: Support Vector Machine 
  # LinearRegression: Basic linear regression
  # MLPRegressor: Neural Network 
  # GaussianProcessRegressor: Kriging model 
  slreg = StackingCVRegressor( \
    regressors=[ \
      SVR(kernel='rbf'), \
      Pipeline([('preprocessing',PolynomialFeatures(1)),('regressor',LinearRegression())]), \
      MLPRegressor(solver='lbfgs', random_state=seed), \
      GaussianProcessRegressor(alpha=0.5, n_restarts_optimizer=4, normalize_y=False, random_state=seed, \
        kernel=Co(40.0, (1e-3, 1e2))*RBF(length_scale=0.45, length_scale_bounds=(1e-2, 1e1))) \
      ], \
    meta_regressor=LinearRegression(fit_intercept=False), cv=5, random_state=seed, n_jobs=-1)
  
  (sample_size, dim) = sample['X'].shape
  
  # Hyper-parameters setting
  # Cherkassky & Ma reference values: C = 117.278, e = 10.333
  hypars_bl = {}
  hypars_bl['svr_r']  = {'C': [60, 120, 240], 'epsilon': [0.1, 1, 10]}
  hypars_bl['nn']     = {'hidden_layer_sizes':[math.floor(f*(sample_size-1)/(dim+2)) for f in [0.50, 0.75, 1.0]], 'activation':['relu','logistic'], }
  
  # Grid building
  hypars_slr = { \
    'svr__C':       hypars_bl['svr_r']['C'], \
    'svr__epsilon': hypars_bl['svr_r']['epsilon'], \
    'mlpregressor__hidden_layer_sizes':hypars_bl['nn']['hidden_layer_sizes'], \
    'mlpregressor__activation':        hypars_bl['nn']['activation'] }
  slr_grid = GridSearchCV(estimator=slreg, param_grid=hypars_slr, cv=5, refit=True, scoring='neg_mean_squared_error', n_jobs=-1)
  
  # Models list
  model_labels = ['SVR-RBF', 'LR','MLP','GP','SLR']
  models = [GridSearchCV(estimator=SVR(kernel='rbf'), param_grid=hypars_bl['svr_r'], cv=5, refit=True, scoring='neg_mean_squared_error'), \
          Pipeline([('preprocessing',PolynomialFeatures(1)),('regressor',LinearRegression())]), \
          GridSearchCV(estimator=MLPRegressor(solver='lbfgs', random_state=seed), param_grid=hypars_bl['nn'], cv=5, refit=True, scoring='neg_mean_squared_error'), \
          GaussianProcessRegressor(alpha=0.5, n_restarts_optimizer=4, normalize_y=False, random_state=seed, \
                                   kernel=Co(40.0, (1e-3, 1e2))*RBF(length_scale=0.45, length_scale_bounds=(1e-2, 1e1))),
          slr_grid
         ]

  # Training models
  for model in models:
    print('{}: '.format(model.__class__.__name__), end='')
    tmp = time.time()    
    # call fit function using python's getattr function
    getattr(model, 'fit')(sample['X'], sample['y'])
    print(' {:0.3}s'.format(time.time()-tmp)) #time.strftime("%M:%S.%f", time.gmtime(time.time()-tmp))))
      
  return { 'models':models, 'model_labels':model_labels }

In [4]:
def evaluate(sample, models_bundle):
  """
  Models evaluation
  
  Input: sample - dictionary with training (X, y) and testing (Xtest, ytest) data
         models_bundle - dictionary with trained base learners and SLR
  Output: dictionary with performance indicators (indicators) and SLR 
          meta-regressor coefficients (coefficients)
  """
  
  models = models_bundle['models']
  model_labels = models_bundle['model_labels']

  #Performance indicators (individuals, SLR)
  answer = { 'Method':[ label for label in model_labels ]}
  # Coefficient of determination
  answer['R2']    = [ r2_score(sample['y'], mod.predict(sample['X'])) for mod in models ]
  # Mean squared error
  answer['MSEtr'] = [ mean_squared_error(sample['y'], mod.predict(sample['X'])) for mod in models ]
  answer['MSEte'] = [ mean_squared_error(sample['ytest'], mod.predict(sample['Xtest'])) for mod in models ]
  # Median absolute error
  answer['MAEtr'] = [ median_absolute_error(sample['y'], mod.predict(sample['X'])) for mod in models ]
  answer['MAEte'] = [ median_absolute_error(sample['ytest'], mod.predict(sample['Xtest'])) for mod in models ]
  indicators = pd.DataFrame(answer)
  
  #Errors (Committee)
  indicators = indicators.append({ 'Method': 'Committee', \
      # Coefficient of determination
      'R2': r2_score(sample['y'], np.average(list(map(lambda x: x.predict(sample['X']), models[0:-1])), axis=0)), \
      # Mean squared error
      'MSEtr': mean_squared_error(sample['y'], np.average(list(map(lambda x: x.predict(sample['X']), models[0:-1])), axis=0)), \
      'MSEte': mean_squared_error(sample['ytest'], np.average(list(map(lambda x: x.predict(sample['Xtest']), models[0:-1])), axis=0)), \
      # Median absolute error
      'MAEtr': median_absolute_error(sample['y'], np.average(list(map(lambda x: x.predict(sample['X']), models[0:-1])), axis=0)), \
      'MAEte': median_absolute_error(sample['ytest'], np.average(list(map(lambda x: x.predict(sample['Xtest']), models[0:-1])), axis=0)), \
    }, ignore_index=True)
  
  print("%25s\t%10s\t%10s\t%10s" % ("Ensemble", "Score (R2)", "MSE(train)", "MSE(test)"))
  for index, row in indicators.iterrows():
    print("%25s\t%10.3f\t%10.3f\t%10.3f" % (row['Method'], row['R2'], row['MSEtr'], row['MSEte']))  

  print("\nParameters of interest")
  print("%25s\t%s" % ("SVR-RBF", models[0].best_params_)) 
  print("%25s\t%s" % ("LR", "Coefs:  [1, x1, x2, x1^2, x1*x2, x2^2] - %s"%(models[1].named_steps['regressor'].coef_)))
  print("%25s\t%s" % ("MLP", models[2].best_params_)) 
  print("%25s\t%s" % ("GP", "Kern: %s - alpha: %s"%(models[3].kernel_, np.mean(models[3].alpha_))))
  print("%25s\t%s" % ("SLR", "Coefs: %s - %s"%([i for i in models[4].best_estimator_.named_regressors], models[4].best_estimator_.meta_regr_.coef_)))
    
  return { 'indicators': indicators, 'coefficients': models[4].best_estimator_.meta_regr_.coef_ }

### Main program

In [ ]:
"""
Define, train and evaluate models, and store performance measures in google drive
"""
    
boots = 100

# Predefined seeds (seeds = np.random.randint(10000, size=boots))
seeds = np.asarray([1003, 1019, 1024, 133, 1396, 1438, 1445, 1447, 1498, 1520, 1534, 1688, 
                    1705, 1712, 1768, 1848, 1953, 2050, 2108, 2148, 2161, 2231, 2232, 2248, 
                    2306, 2389, 2489, 2492, 2524, 2592, 2707, 2710, 2779, 3044, 3098, 32, 
                    3284, 3329, 3361, 3372, 3576, 3603, 3610, 363, 3632, 372, 3742, 3744, 
                    383, 3935, 4019, 4237, 4272, 4363, 4464, 4604, 4718, 4721, 4834, 484, 
                    4884, 489, 4891, 4926, 4932, 5035, 5066, 5106, 5276, 545, 5525, 5660, 
                    5686, 5703, 5784, 5813, 5838, 5925, 5994, 6081, 6180, 6316, 6400, 6429, 
                    6523, 6542, 6557, 6655, 6664, 6688, 684, 6972, 7016, 7061, 7161, 7170, 
                    7188, 7190, 7201, 7309, 7328, 7341, 7391, 754, 7548, 7713, 7749, 7756, 
                    7784, 7818, 7895, 8011, 8021, 8055, 8060, 8073, 8106, 8127, 8145, 8374, 
                    8434, 8466, 8487, 8536, 858, 860, 8675, 8689, 8839, 8849, 8882, 8896, 
                    8981, 9026, 9056, 9160, 9225, 9278, 9306, 9364, 9394, 9468, 9711, 9820, 
                    9834, 9859, 9936, 9981, 2403, 1904]) 

# Profiling
start_time = time.time()

# Auto-iterate through all files in the root folder.
file_list = [file for file in os.listdir('.') if os.path.isfile(file)]
print('Files on disk: {}'.format(len(file_list)))
   
# Main cycle
for cur_seed in seeds[0:boots]:
  filename = 'fsae_von_180_curseed_{}.pkl'.format(cur_seed)
  matching_file = [ match for match in file_list if match == filename ]  # Get already simulated seeds
  if len(matching_file) > 0:
    # Seed already simulated. Skip to next cycle.
    print('[%s]Seed %d ready!'% (time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time)),cur_seed)) 
    continue
  
  np.random.seed(cur_seed)
  random.seed(cur_seed)
  
  # Loading train and test sets
  print("[{}]Loading data... ".format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))
  sample = load_data()
  
  # Train individual models and SLR
  print("[{}]Training models...".format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))
  models_bundle = model_train(cur_seed, sample)
  
  # Calculate and print performance measures
  print("[{}]Calculating performance measures...".format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))
  evaluation = evaluate(sample, models_bundle)
  
  print('[%s]Seed %d ready!'% (time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time)),cur_seed), end='')
  print('%d of %d total seeds.' % (seeds.tolist().index(cur_seed)+1, len(seeds)))
  
  data = {'seed':cur_seed, 'sample':sample, 'models_bundle':models_bundle, 'evaluation':evaluation }
  
  # Dumping state to disk...
  print('[{}]Dumping to local disk file: {} ...'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time)),filename))
  with open(filename, 'wb') as f:  
    pickle.dump([data], f)
  
  print('DONE!')
  continue

print("DONE!")
print ("All seeds evaluated!!")

Files on disk: 55
[00:00:00]Seed 1003 ready!
[00:00:00]Seed 1019 ready!
[00:00:00]Seed 1024 ready!
[00:00:00]Seed 133 ready!
[00:00:00]Seed 1396 ready!
[00:00:00]Seed 1438 ready!
[00:00:00]Seed 1445 ready!
[00:00:00]Seed 1447 ready!
[00:00:00]Seed 1498 ready!
[00:00:00]Seed 1520 ready!
[00:00:00]Seed 1534 ready!
[00:00:00]Seed 1688 ready!
[00:00:00]Seed 1705 ready!
[00:00:00]Seed 1712 ready!
[00:00:00]Seed 1768 ready!
[00:00:00]Seed 1848 ready!
[00:00:00]Seed 1953 ready!
[00:00:00]Seed 2050 ready!
[00:00:00]Seed 2108 ready!
[00:00:00]Seed 2148 ready!
[00:00:00]Seed 2161 ready!
[00:00:00]Seed 2231 ready!
[00:00:00]Seed 2232 ready!
[00:00:00]Seed 2248 ready!
[00:00:00]Seed 2306 ready!
[00:00:00]Seed 2389 ready!
[00:00:00]Seed 2489 ready!
[00:00:00]Seed 2492 ready!
[00:00:00]Seed 2524 ready!
[00:00:00]Seed 2592 ready!
[00:00:00]Seed 2707 ready!
[00:00:00]Seed 2710 ready!
[00:00:00]Seed 2779 ready!
[00:00:00]Seed 3044 ready!
[00:00:00]Seed 3098 ready!
[00:00:00]Seed 32 ready!
[00:00:00]See

### Visualization (individual model)

In [ ]:
"""
Visualization of results
"""

# Google drive: Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Google drive: Reading available files on disk
file_list = [file for file in os.listdir('.') if os.path.isfile(file)]
print('Files on disk: {}'.format(len(file_list)))

# PICKLE files
file_pattern = 'fsae_von_180_'

file_targets = [ match for match in file_list if match.startswith(file_pattern) ]  # Get already simulated seeds
print('Target files: {}'.format(len(file_targets)))
unified_data = pd.DataFrame()
unified_coef = []

win_freq = []
names = ['SVR-RBF','LR','MLP','GP','Committee','SLR']
for file in file_targets: 
  with open(file['title'],'rb') as f:
    data = pickle.load(f)[0]
  data['evaluation'] = evaluate(data['sample'],data['models_bundle'])  
  evaluation = data['evaluation']
  unified_data = unified_data.append(data['evaluation']['indicators']) 
  unified_coef.append(data['evaluation']['coefficients'])
  unified_data = unified_data.append(data['evaluation']['indicators']) 
  # Performance measures unification
  # Coefficient of determination
  r2s = [ r2_score(data['sample']['y'], cur.predict(data['sample']['X'])) for cur in data['models_bundle']['models'] ]
  r2s[4:4] = [r2_score(data['sample']['y'], np.average(list(map(lambda x: x.predict(data['sample']['X']), data['models_bundle']['models'][0:-1])), axis=0))]
  # Mean squared error
  mses = [ mean_squared_error(data['sample']['ytest'], cur.predict(data['sample']['Xtest'])) for cur in data['models_bundle']['models'] ]
  mses[4:4] = [mean_squared_error(data['sample']['ytest'], np.average(list(map(lambda x: x.predict(data['sample']['Xtest']), data['models_bundle']['models'][0:-1])), axis=0))]
  # Median absolute error
  maes = [ median_absolute_error(data['sample']['ytest'], cur.predict(data['sample']['Xtest'])) for cur in data['models_bundle']['models'] ]
  maes[4:4] = [median_absolute_error(data['sample']['ytest'], np.average(list(map(lambda x: x.predict(data['sample']['Xtest']), data['models_bundle']['models'][0:-1])), axis=0))]
  # Winner model (smaller MSE) identification
  win_idx = np.where(mses == np.amin(mses))
  win_freq.append(names[win_idx[0][0]])
  # Specific seed visualization
  if data['seed'] == 1024:
    print('Weights for seed {}: {}'.format(data['seed'], evaluation['coefficients']))    
    print(evaluation['indicators'])
        
# Set (te | tr)
theSet = "te"
# The indicator (MSE | MAE)
theIndic = "MSE"
# Indicator limit (sharing y-scales in continuous boxplots)
maxY = 20    # MAE Testing
# maxY = 2500  # MSE Testing
# maxY = 700   # MSE Training

toPlot = theIndic+theSet
print('To plot: {}'.format(toPlot))

tmp_palette = sns.color_palette(n_colors=5)
tmp_palette.append((0.3, 0.6, 0.9))


fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10), gridspec_kw={'width_ratios': [4, 1, 1]})#, 'height_ratios':[3, 1.5]})
sns.boxplot(data=unified_data.query('Method in ["SVR-RBF","LR","MLP","GP"]'), x='Method', y=toPlot, notch=True, ax=axes[0][0], width=0.5, palette=tmp_palette)
axes[0][0].set_ylabel('MSE')
axes[0][0].set_xlabel(' ')
axes[0][0].set_title('Individual surrogates')
axes[0][0].set_ylim(0, maxY)#SLR-CV
sns.boxplot(data=unified_data.query('Method == "Committee"'), x='Method', y=toPlot, notch=True, ax=axes[0][1], color=sns.color_palette(n_colors=5)[-1], width=0.5)
axes[0][1].set_ylabel('MSE')
axes[0][1].set_xlabel(' ')
axes[0][1].set_title("Committee")
axes[0][1].set_ylim(0, maxY)#SLR-CV
axes[0][1].axes.get_yaxis().set_ticklabels([])
axes[0][1].axes.get_xaxis().set_ticklabels([])
sns.boxplot(data=unified_data.query('Method == "SLR"'), x='Method', y=toPlot, notch=True, ax=axes[0][2], color=(0.3, 0.6, 0.9), width=0.5)
axes[0][2].set_ylabel('MSE')
axes[0][2].set_xlabel(' ')
axes[0][2].set_title('SLR')
axes[0][2].set_ylim(0, maxY)#SLR-CV
axes[0][2].axes.get_yaxis().set_ticklabels([])
axes[0][2].axes.get_xaxis().set_ticklabels([])

sns.boxplot(x="variable",y="value",data=pd.melt(pd.DataFrame(unified_coef, columns=['SVR','LR ','MLP','GP'])), notch=True, width=0.4, ax=axes[1][0], palette=tmp_palette)
axes[1][0].set_xlabel('Meta-regressor coefficients')
axes[1][0].set_ylabel('Coefficient value')
axes[1][0].axes.get_xaxis().set_ticklabels([])

axes[1][0].set_xlabel('Meta-regressor coefficients')

f, ax1 = plt.subplots(figsize=(5.3, 3))
freqs = [ win_freq.count(X) for X in names]
sns.barplot(data=pd.DataFrame({'models':names,'Frequency':freqs}), x='models', y='Frequency', palette=tmp_palette)

print("\nMSE\tTrain\tTest\nCommittee\t{:9.3f}\t{:9.3f}\nSLR\t{:9.3f}\t{:9.3f}".format( \
   np.median(unified_data[unified_data.Method == 'Committee']['MSEtr']), \
   np.median(unified_data[unified_data.Method == 'Committee']['MSEte']), \
   np.median(unified_data[unified_data.Method == 'SLR']['MSEtr']), \
   np.median(unified_data[unified_data.Method == 'SLR']['MSEte'])))

